In [1]:
import tensorflow as tf

2023-06-27 11:44:18.565308: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-27 11:44:18.588222: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-27 11:44:18.928667: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from tensorflow import keras

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
data=pd.read_csv('./torch-v1.8.0.csv')

In [5]:
data.head()

,API Name,Parameters,Test case,Label
0,torch.nn.functional.max_pool3d,"arg_0_tensor = torch.rand([20, 16, 50, 44, 31]...",v1.8.0/cuda-oracle/success/torch.nn.functional...,success
1,torch.nn.functional.max_pool3d,"arg_0_tensor = torch.rand([20, 16, 50, 44, 31]...",v1.8.0/cuda-oracle/success/torch.nn.functional...,success
2,torch.nn.functional.max_pool3d,"arg_0_tensor = torch.rand([20, 16, 51, 33, 15]...",v1.8.0/cuda-oracle/success/torch.nn.functional...,success
3,torch.nn.functional.max_pool3d,"arg_0_tensor = torch.rand([20, 16, 51, 33, 15]...",v1.8.0/FreeFuzz_bugs/torch.nn.functional.max_p...,bug
4,torch.isneginf,"arg_0_tensor = torch.rand([3], dtype=torch.flo...",v1.8.0/cuda-oracle/success/torch.isneginf/1.py,success


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44143 entries, 0 to 44142
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   API Name    44143 non-null  object
 1   Parameters  41096 non-null  object
 2   Test case   44143 non-null  object
 3   Label       44143 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


In [7]:
data.iloc[:,-1].value_counts()

success    42666
bug         1477
Name: Label, dtype: int64

In [8]:
data.iloc[:, -2].head()

0    v1.8.0/cuda-oracle/success/torch.nn.functional...
1    v1.8.0/cuda-oracle/success/torch.nn.functional...
2    v1.8.0/cuda-oracle/success/torch.nn.functional...
3    v1.8.0/FreeFuzz_bugs/torch.nn.functional.max_p...
4       v1.8.0/cuda-oracle/success/torch.isneginf/1.py
Name: Test case, dtype: object

In [9]:
data.iloc[:, -1].replace("success",1).replace("bug",0).head()

0    1
1    1
2    1
3    0
4    1
Name: Label, dtype: int64

In [10]:
data['combintedString']=data[['API Name','Parameters']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)

In [11]:
data.head()

,API Name,Parameters,Test case,Label,combintedString
0,torch.nn.functional.max_pool3d,"arg_0_tensor = torch.rand([20, 16, 50, 44, 31]...",v1.8.0/cuda-oracle/success/torch.nn.functional...,success,torch.nn.functional.max_pool3darg_0_tensor = t...
1,torch.nn.functional.max_pool3d,"arg_0_tensor = torch.rand([20, 16, 50, 44, 31]...",v1.8.0/cuda-oracle/success/torch.nn.functional...,success,torch.nn.functional.max_pool3darg_0_tensor = t...
2,torch.nn.functional.max_pool3d,"arg_0_tensor = torch.rand([20, 16, 51, 33, 15]...",v1.8.0/cuda-oracle/success/torch.nn.functional...,success,torch.nn.functional.max_pool3darg_0_tensor = t...
3,torch.nn.functional.max_pool3d,"arg_0_tensor = torch.rand([20, 16, 51, 33, 15]...",v1.8.0/FreeFuzz_bugs/torch.nn.functional.max_p...,bug,torch.nn.functional.max_pool3darg_0_tensor = t...
4,torch.isneginf,"arg_0_tensor = torch.rand([3], dtype=torch.flo...",v1.8.0/cuda-oracle/success/torch.isneginf/1.py,success,"torch.isneginfarg_0_tensor = torch.rand([3], d..."


In [12]:
x=data.iloc[:, -1]
y=data.iloc[:, -2].replace("success",1).replace("bug",0)

In [13]:
x.shape

(44143,)

In [14]:
import re

In [15]:
token = re.compile('[A-Za-z]+|[!?,.()]')

In [16]:
def reg_text(text):
    new_text = token.findall(text)
    new_text = [word.lower() for word in new_text]
    return new_text

In [17]:
data['text'] = data.combintedString.apply(reg_text)

In [18]:
word_set = set()
for text in data.text:
    for word in text:
        word_set.add(word) 

In [19]:
max_word = len(word_set) + 1
max_word

508

In [20]:
word_list = list(word_set)

In [21]:
word_list.index('arg')

448

In [22]:
word_index =  dict((word, word_list.index(word) + 1) for word in word_list)

In [23]:
word_index

{'pixelshufflearg': 1,
 'celunan': 2,
 'trilarg': 3,
 'matmularg': 4,
 'b': 5,
 'crossentropylossnan': 6,
 'reluarg': 7,
 'logdetarg': 8,
 'stacklevel': 9,
 'block': 10,
 'randint': 11,
 'constant': 12,
 'dscale': 13,
 'minarg': 14,
 'unique': 15,
 'bn': 16,
 'lstmcellarg': 17,
 'unbindarg': 18,
 'bcelossnan': 19,
 'reduction': 20,
 'parameters': 21,
 'sumarg': 22,
 'exparg': 23,
 'bm': 24,
 'e': 25,
 'baddbmmarg': 26,
 'xorarg': 27,
 'sinharg': 28,
 'is': 29,
 'bilinear': 30,
 'tanhshrinkarg': 31,
 'layers': 32,
 'marginrankinglossnan': 33,
 'constantpad': 34,
 'isnanarg': 35,
 'num': 36,
 'medianarg': 37,
 '(': 38,
 'compute': 39,
 'tril': 40,
 'conjarg': 41,
 'logaddexparg': 42,
 'response': 43,
 'rand': 44,
 'ravelarg': 45,
 'diagarg': 46,
 'selunan': 47,
 'bernoulliarg': 48,
 'negarg': 49,
 'int': 50,
 'generator': 51,
 'replicate': 52,
 'keepdim': 53,
 'groupnormarg': 54,
 'high': 55,
 'diagonalarg': 56,
 'rot': 57,
 'qrarg': 58,
 'igammaarg': 59,
 'batch': 60,
 'signbitarg': 61,

In [24]:
data_ok = data.text.apply(lambda x: [word_index.get(word, 0) for word in x])

In [25]:
len(data_ok.iloc[2])

39

In [26]:
maxlen = max(len(x) for x in data_ok)

In [27]:
maxlen

14262

In [28]:
data_ok = keras.preprocessing.sequence.pad_sequences(data_ok.values, maxlen=maxlen)

In [29]:
data_ok.shape

(44143, 14262)

AttributeError: 'Series' object has no attribute 'review'

In [31]:
model = tf.keras.Sequential()

2023-06-27 11:44:36.030273: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-27 11:44:36.415462: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-27 11:44:36.415561: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [32]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_word, 50, input_length=maxlen))
model.add(tf.keras.layers.Conv1D(32, 7, activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling1D(3))
model.add(tf.keras.layers.Conv1D(32, 7, activation='relu', padding='same'))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 14262, 50)         25400     
                                                                 
 conv1d (Conv1D)             (None, 14262, 32)         11232     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 4754, 32)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 4754, 32)          7200      
                                                                 
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                

In [34]:
model.compile(optimizer=keras.optimizers.RMSprop(),loss='binary_crossentropy',metrics=['acc'])#accuricy

In [35]:
history=model.fit(data_ok,y,epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10


2023-06-27 11:44:50.098395: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-27 11:44:51.730151: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-27 11:44:51.733295: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f422c021130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-27 11:44:51.733369: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-06-27 11:44:51.777899: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-27 11:44:52.068143: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

276/276 [==============================] - 22s 57ms/step - loss: 0.1621 - acc: 0.9662 - val_loss: 0.1930 - val_acc: 0.9537
Epoch 2/10
276/276 [==============================] - 12s 43ms/step - loss: 0.1368 - acc: 0.9698 - val_loss: 0.1897 - val_acc: 0.9537
Epoch 3/10
276/276 [==============================] - 12s 43ms/step - loss: 0.1366 - acc: 0.9698 - val_loss: 0.1896 - val_acc: 0.9537
Epoch 4/10
276/276 [==============================] - 12s 43ms/step - loss: 0.1365 - acc: 0.9698 - val_loss: 0.1935 - val_acc: 0.9537
Epoch 5/10
276/276 [==============================] - 12s 42ms/step - loss: 0.1367 - acc: 0.9698 - val_loss: 0.2018 - val_acc: 0.9537
Epoch 6/10
276/276 [==============================] - 12s 42ms/step - loss: 0.1367 - acc: 0.9698 - val_loss: 0.1875 - val_acc: 0.9537
Epoch 7/10
276/276 [==============================] - 12s 43ms/step - loss: 0.1364 - acc: 0.9698 - val_loss: 0.1914 - val_acc: 0.9537
Epoch 8/10
276/276 [==============================] - 12s 42ms/step - los

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.epoch,history.history.get('loss'))

In [ ]:
model.evaluate

In [ ]:
#for test data

In [36]:
test_data=pd.read_csv('./torch-testdata.csv')

In [39]:
test_data['combintedString']=test_data[['API Name','Parameters']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)

In [40]:
test_data['text'] = test_data.combintedString.apply(reg_text)

In [41]:
data_test_ok = test_data.text.apply(lambda x: [word_index.get(word, 0) for word in x])

In [42]:
data_test_ok = keras.preprocessing.sequence.pad_sequences(data_test_ok.values, maxlen=maxlen)

In [44]:
predict=model.predict(data_test_ok)

3/3 [==============================] - 0s 3ms/step


In [45]:
predict

array([[0.9635039 ],
       [0.96348554],
       [0.96349293],
       [0.96376616],
       [0.9636483 ],
       [0.96356344],
       [0.96349585],
       [0.9636045 ],
       [0.96378917],
       [0.9636434 ],
       [0.96356344],
       [0.96356344],
       [0.96349794],
       [0.96351683],
       [0.9635655 ],
       [0.96357906],
       [0.9635027 ],
       [0.9635587 ],
       [0.9636115 ],
       [0.96355814],
       [0.9635527 ],
       [0.96355826],
       [0.96349496],
       [0.9635655 ],
       [0.96365076],
       [0.9635702 ],
       [0.9637082 ],
       [0.9634915 ],
       [0.96366686],
       [0.96355915],
       [0.96381706],
       [0.9636483 ],
       [0.96357286],
       [0.9635745 ],
       [0.96349794],
       [0.96358705],
       [0.9634136 ],
       [0.9636023 ],
       [0.96385574],
       [0.96363807],
       [0.96363807],
       [0.9635725 ],
       [0.96359324],
       [0.96357393],
       [0.9636478 ],
       [0.96368223],
       [0.96349174],
       [0.963

In [62]:
bug_count=0
nonbug_count=0
for numbers in predict:
    if numbers[0]>0.9635:
        nonbug_count+=1
    else:
        bug_count+=1
print(bug_count)

22


In [64]:
labels = []

for prob in predict:
    if prob >= 0.9635:
        labels.append("success")
    else:
        labels.append("fail")
test_data["Label"]=labels
test_data.to_csv('./outputdata.csv')